## Dimensioning function

The following funtion is used for dimensioning step, where you have to provide the all the 3 views images and the output of this function will be the image highlighted with a line and the unit length to pixel value. Once you get the raito value you can multiple it with the length of the highlighted line to get the length to pixel ratio for that view.

In [ ]:
import copy
import json
import os
import shutil
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Image
from google.colab.patches import cv2_imshow
from google.colab import files

In [ ]:
uploaded = files.upload()


Saving front.jpg to front.jpg


In [ ]:
for fname in uploaded.keys():
    image_path = fname

In [ ]:
img = cv2.imread(image_path)
orig = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

In [ ]:
# Find contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
hierarchy, contours = zip(*sorted(zip(hierarchy[0], contours), key=lambda x: cv2.contourArea(x[1]), reverse=True))

shapes_detected = []

In [ ]:
# Iterate and detect shape + dimensions
for i, c in enumerate(contours):
    if hierarchy[i][3] != -1 or (hierarchy[i][3] == -1 and hierarchy[i][2] == -1):
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box.astype(int)
            x, y, w, h = cv2.boundingRect(c)

            # Approximate shape
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.04 * peri, True)
            num_vertices = len(approx)
            shape = "unidentified"

            if num_vertices == 3:
                shape = "triangle"
            elif num_vertices == 4:
                aspect_ratio = w / float(h)
                shape = "square" if 0.95 <= aspect_ratio <= 1.05 else "rectangle"
            elif num_vertices == 5:
                shape = "pentagon"
            elif num_vertices > 5:
                shape = "circle"

            cv2.drawContours(orig, [box], 0, (0, 0, 255), 2)
            cv2.putText(orig, f"{shape} {w}x{h}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            shapes_detected.append((shape, w, h))
            break  # only the biggest outer shape for this demo


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2546: error: (-215:Assertion failed) npoints > 0 in function 'drawContours'


In [ ]:
cv2_imshow(orig)
plt.title("Detected Shape and Dimensions")
plt.show()

print("\nShapes Detected (Shape, Width, Height):")
for s in shapes_detected:
    print(s)
